In [2]:

import pandas as pd
import numpy as np
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Mock function for yfinance data since we can't use yfinance here
def generate_mock_data():
    np.random.seed(42)
    dates = pd.date_range(end=pd.Timestamp.today(), periods=60)
    data = {
        'Close': np.cumsum(np.random.randn(60)) + 150  # Simulated closing prices
    }
    df = pd.DataFrame(data, index=dates)
    df['Lag1'] = df['Close'].shift(1)
    df['Lag2'] = df['Close'].shift(2)
    df['MA5'] = df['Close'].rolling(5).mean()
    df['MA10'] = df['Close'].rolling(10).mean()
    df.dropna(inplace=True)
    return df[['Lag1', 'Lag2', 'MA5', 'MA10']], df['Close'].loc[df.index], df.index

tickers = ['NVDA', 'AAPL', 'MSFT', 'AMZN']
results = []

# Loop through each ticker to simulate modeling process
for ticker in tickers:
    X, y, dates = generate_mock_data()

    # Add noise to simulate randomness
    X_noise = X + np.random.normal(0, 0.001, X.shape)

    # Bayesian Ridge Regression and parameter search
    model = BayesianRidge()
    param_dist = {
        "alpha_1": np.logspace(-6, -2, 50),
        "lambda_1": np.logspace(-6, -2, 50)
    }
    tscv = TimeSeriesSplit(n_splits=5)
    search = RandomizedSearchCV(model, param_distributions=param_dist,
                                n_iter=10, cv=tscv, scoring='neg_mean_squared_error', random_state=42)
    search.fit(X_noise, y)
    best_model = search.best_estimator_

    # Predictions with standard deviation
    y_pred, y_std = best_model.predict(X, return_std=True)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    last_actual = y.iloc[-1]
    last_pred = y_pred[-1]
    uncertainty = y_std[-1]

    expected_return = (last_pred - last_actual) / last_actual
    risk_adjusted_score = expected_return / uncertainty if uncertainty != 0 else 0

    results.append({
        'Ticker': ticker,
        'Expected Return': expected_return,
        'Uncertainty': uncertainty,
        'Risk-Adjusted Score': risk_adjusted_score,
        'RMSE': rmse,
        'Model': best_model
    })

# Create DataFrame from results and sort
ranked = pd.DataFrame(results).sort_values(by='Risk-Adjusted Score', ascending=False)
ranked[['Ticker', 'Expected Return', 'Uncertainty', 'Risk-Adjusted Score', 'RMSE']]

<ipython-input-2-865c40c71fb9>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  uncertainty = y_std[-1]
<ipython-input-2-865c40c71fb9>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  uncertainty = y_std[-1]
<ipython-input-2-865c40c71fb9>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  uncertainty = y_std[-1]
<ipython-input-2-865c40c71fb9>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futur

,Ticker,Expected Return,Uncertainty,Risk-Adjusted Score,RMSE
0,NVDA,-0.003462,3.3636,-0.001029,0.738979
1,AAPL,-0.003462,3.3636,-0.001029,0.738979
2,MSFT,-0.003462,3.3636,-0.001029,0.738979
3,AMZN,-0.003462,3.3636,-0.001029,0.738979


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Ticker').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Expected Return']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Expected Return'}, axis=1)
              .sort_values('Expected Return', ascending=True))
  xs = counted['Expected Return']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('Expected Return', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Ticker')):
  _plot_series(series, series_name, i)
  fig.legend(title='Ticker', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Expected Return')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Uncertainty']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Uncertainty'}, axis=1)
              .sort_values('Uncertainty', ascending=True))
  xs = counted['Uncertainty']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('Uncertainty', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Ticker')):
  _plot_series(series, series_name, i)
  fig.legend(title='Ticker', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Uncertainty')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Uncertainty']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Uncertainty'}, axis=1)
              .sort_values('Uncertainty', ascending=True))
  xs = counted['Uncertainty']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_9.sort_values('Uncertainty', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Ticker')):
  _plot_series(series, series_name, i)
  fig.legend(title='Ticker', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Uncertainty')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Risk-Adjusted Score']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Risk-Adjusted Score'}, axis=1)
              .sort_values('Risk-Adjusted Score', ascending=True))
  xs = counted['Risk-Adjusted Score']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_10.sort_values('Risk-Adjusted Score', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Ticker')):
  _plot_series(series, series_name, i)
  fig.legend(title='Ticker', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Risk-Adjusted Score')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_11['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_12['Ticker'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_12, x='index', y='Ticker', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)